In [4]:
import altair as alt
import panel as pn

from vega_datasets import data


pn.extension("vega", sizing_mode="stretch_width", template="fast")

In [5]:
def get_theme():
    return pn.state.session_args.get("theme", [b'default'])[0].decode()
theme=get_theme()
theme

'default'

In [6]:
nice_accent_colors = [
    ("#00A170", "white"), # Mint
    ("#DAA520", "white"), # Golden Rod
    ("#F08080", "white"), # Light Coral
    ("#4099da", "white"), # Summery Sky
    ("#2F4F4F", "white"), # Dark Slate Grey
    ("#A01346", "white"), # Fast
]

In [7]:
def get_nice_accent_color():
    """Returns the 'next' nice accent color"""
    if not "color_index" in pn.state.cache:
        pn.state.cache["color_index"]=0
    elif pn.state.cache["color_index"]==len(nice_accent_colors)-1:
        pn.state.cache["color_index"]=0
    else:
        pn.state.cache["color_index"]+=1
    return nice_accent_colors[pn.state.cache["color_index"]]

In [8]:
accent_color, color = get_nice_accent_color()
pn.pane.Markdown(f"# Color: {accent_color}", background=accent_color, height=70, margin=0, style={"color": color, "padding": "10px"})

Markdown(str, background='#00A170', height=70, margin=0, sizing_mode='stretch_width', style={'color': 'white', ...})

In [10]:
def get_vega_plot(theme="default", accent_color="blue"):
    vegalite = {
      "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
      "description": "A simple bar chart with rounded corners at the end of the bar.",
      "width": "container",
      "height": "container",
      "data": {
        "values": [
          {"a": "A", "b": 28},
          {"a": "B", "b": 55},
          {"a": "C", "b": 43},
          {"a": "D", "b": 91},
          {"a": "E", "b": 81},
          {"a": "F", "b": 53},
          {"a": "G", "b": 19},
          {"a": "H", "b": 87},
          {"a": "I", "b": 52}
        ]
      },
      "mark": {"type": "bar", "cornerRadiusEnd": 4, "tooltip": True},
      "encoding": {
        "x": {"field": "a", "type": "ordinal"},
        "y": {"field": "b", "type": "quantitative"},
        "color": {"value": accent_color},
      }
    }

    if theme == "dark":
        vegalite["config"] = {
            "background": "#333",
            "title": {"color": "#fff"},
            "style": {"guide-label": {"fill": "#fff"}, "guide-title": {"fill": "#fff"}},
            "axis": {"domainColor": "#fff", "gridColor": "#888", "tickColor": "#fff"},
        }
    return vegalite

In [11]:
vega_plot = get_vega_plot(theme=theme, accent_color=accent_color)
vega_pane = pn.pane.Vega(vega_plot, height=500, sizing_mode="stretch_both", name="VEGA")
vega_pane

Vega(dict, height=500, name='VEGA', selection=Selection, sizing_mode='stretch_both')

In [12]:
def get_altair_plot(theme="default", accent_color="blue"):
    if theme == "dark":
        alt.themes.enable("dark")
    else:
        alt.themes.enable("default")
    return (
        alt.Chart(data.cars())
        .mark_circle(size=200)
        .encode(
            x='Horsepower:Q',
            y='Miles_per_Gallon:Q',
            tooltip=["Name", "Origin", "Horsepower", "Miles_per_Gallon"],
        )
        .configure_mark(
            color=accent_color
        )
        .properties(
            height="container",
            width="container",
        )
        .interactive()
    )

In [13]:
altair_plot = get_altair_plot(theme=theme, accent_color=accent_color)
altair_pane = pn.pane.Vega(altair_plot, height=500, sizing_mode="stretch_both", name="ALTAIR")
altair_pane

Vega(Chart, height=500, name='ALTAIR', selection=Selection, sizing_mode='stretch_both')

In [17]:
pn.Tabs(vega_pane, altair_pane).servable(title="Panel - Vega/ Altair with custom styling")

Tabs(sizing_mode='stretch_width')
    [0] Vega(dict, height=500, name='VEGA', selection=Selection, sizing_mode='stretch_both')
    [1] Vega(Chart, height=500, name='ALTAIR', selection=Selection, sizing_mode='stretch_both')

In [15]:
pn.state.template.param.update(accent_base_color=accent_color, header_background=accent_color)